<a href="https://colab.research.google.com/github/Elkkujou/Gradu/blob/main/Regiimi_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# # GitHub Repository Setup
#
# This cell navigates to `/content`, removes any previous clone of the repository,
# clones the latest version from GitHub, and lists the repository files.

# %%
%cd /content
!rm -rf Gradu
!git clone https://github.com/Elkkujou/Gradu.git
%cd /content/Gradu
!ls


/content
Cloning into 'Gradu'...
remote: Enumerating objects: 189, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 189 (delta 22), reused 0 (delta 0), pack-reused 136 (from 1)
Receiving objects: 100% (189/189), 19.67 MiB | 9.72 MiB/s, done.
Resolving deltas: 100% (80/80), done.
/content/Gradu
 chatti_RF.ipynb	       regime_pred.txt			'RF_regime (3).ipynb'
'Financial turbulence.ipynb'   RF_Gradu.ipynb			 THE_ONE.xlsx
 FT_source.xlsx		      'RF REGIIMI HYVÄ TRAINING.ipynb'


In [ ]:
skip_training = False
use_regime_split = True


In [ ]:
# # Import Required Libraries
#
# Import all necessary libraries for data manipulation, visualization,
# machine learning, and regression analysis.

# %%
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm
from tabulate import tabulate

from IPython.display import display, HTML


In [ ]:
# Load and flatten the data
xls_file = pd.ExcelFile("/content/Gradu/THE_ONE.xlsx")
df = xls_file.parse("returns non-log")
df.columns = df.columns.get_level_values(0)

print("Headers in the 'returns non-log' sheet:")
print(df.columns)

# Features to used as predictive variables

#UNUSED: 'Financial Turbulance'
# 'H.Regimes' // TÄÄÄ RIKKI, EI OO NUMERO

FEATURES = ['VIXCLS', 'CPI', 'LEI', '10yr', 'Financial condition', 'Drawdown', '10y-2y', 'Financial Turbulance']

# Model split by column
REGIMES_COLUMN = 'Regimes'

# Used factors
FACTORS = [
    'USA MOMENTUM Standard (Large+Mid Cap)',
    'USA MINIMUM VOLATILITY (USD) Standard (Large+Mid Cap)',
    'USA RISK WEIGHTED Standard (Large+Mid Cap)',
    'USA SECTOR NEUTRAL QUALITY Standard (Large+Mid Cap)',
    'USA ENHANCED VALUE Standard (Large+Mid Cap)'
]


BENCHMARK = [
    'USA Standard (Large+Mid Cap)'
]

# Convert the leftmost column (assumed to be the date column) to datetime
date_column = df.columns[0]
df[date_column] = pd.to_datetime(df[date_column])

# Retrieve first and last observation dates and count observations
first_date = df[date_column].iloc[0]
last_date = df[date_column].iloc[-1]
n_observations = len(df)

# Create a DataFrame with the information
info_df = pd.DataFrame({
    "Description": ["First observation date", "Last observation date", "Total number of observations"],
    "Value": [first_date, last_date, n_observations]
})

# Display the results as a neat HTML table
display(HTML(info_df.to_html(index=False, classes="table table-striped", border=0)))



Headers in the 'returns non-log' sheet:
Index(['Date', 'USA Standard (Large+Mid Cap)',
       'USA MOMENTUM Standard (Large+Mid Cap)',
       'USA MINIMUM VOLATILITY (USD) Standard (Large+Mid Cap)',
       'USA RISK WEIGHTED Standard (Large+Mid Cap)',
       'USA SECTOR NEUTRAL QUALITY Standard (Large+Mid Cap)',
       'USA ENHANCED VALUE Standard (Large+Mid Cap)', 'Winning Factor',
       'VIXCLS', 'GDPC1', 'CPI', 'LEI', '10yr', 'Financial Turbulance',
       'Regimes', 'Financial condition', 'Drawdown', 'HY Spread', 'IG Spread',
       '10y-2y', 'Vix-regimes2', 'Vix-regimes3', 'FT_regimes2', 'FT_regimes3',
       'DD_regimes2', 'DD_regimes3', 'cluster_0 (minimal drawdown',
       'cluster_1 (moderate drawdown)', 'cluster_2 (severe drawdown)', 'SMB',
       'HML', 'RMW', 'CMA'],
      dtype='object')


Description,Value
First observation date,1990-03-30 00:00:00
Last observation date,2024-10-31 00:00:00
Total number of observations,416


In [ ]:
# Check if 'regime' exists; if not, create it from the regime indicator columns.
if 'regime' not in df.columns:
    regime_cols = ['cluster_0 (minimal drawdown', 'cluster_1 (moderate drawdown)', 'cluster_2 (severe drawdown)']
    df['regime'] = df[regime_cols].idxmax(axis=1).str.extract(r'cluster_(\d)')[0].astype(int)

['VIXCLS', 'CPI', 'LEI', '10yr', 'Financial condition', '10y-2y', 'Financial Turbulance']

In [ ]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline  # Use imblearn's pipeline for SMOTE compatibility
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

# Parameters:
overall_window_size = 155   # Number of rows used as training data
min_data_threshold = 4      # Minimum training observations required

# Define features used for regime prediction.
FEATURES_regime = ['VIXCLS', 'CPI', 'LEI', '10yr', 'Financial condition', '10y-2y', 'Financial Turbulance']
target_regime = 'regime'

# Optional: If you want to filter training by regime:
use_regime_split = False
REGIMES_COLUMN = 'regime'

# Ensure that the 'regime' target column exists.
regime_cols = ['cluster_0 (minimal drawdown', 'cluster_1 (moderate drawdown)', 'cluster_2 (severe drawdown)']
if 'regime' not in df.columns:
    df['regime'] = df[regime_cols].idxmax(axis=1).str.extract(r'cluster_(\d)')[0].astype(int)

# Sort DataFrame by Date.
df_sorted = df.sort_values('Date').reset_index(drop=True)

results_regime = []
feature_importances_list = []  # To accumulate RF feature importances

for i in range(overall_window_size, len(df_sorted)):
    test_row = df_sorted.iloc[i]
    predicted_month = test_row['Date']

    if use_regime_split:
        test_regime = test_row[REGIMES_COLUMN]
    else:
        test_regime = None

    overall_train_window = df_sorted.iloc[i - overall_window_size : i]
    if use_regime_split:
        regime_train_window = overall_train_window[overall_train_window[REGIMES_COLUMN] == test_regime]
    else:
        regime_train_window = overall_train_window

    if len(regime_train_window) < min_data_threshold:
        print(f"Test row {predicted_month.date()}: insufficient obs (<{min_data_threshold}), skipping.")
        continue

    last_train_date = regime_train_window['Date'].iloc[-1]
    if last_train_date.year == predicted_month.year and last_train_date.month >= predicted_month.month:
        print(f"Test row {predicted_month.date()}: last training date {last_train_date.date()} in same month, skipping.")
        continue

    train_start_date = regime_train_window['Date'].iloc[0]
    train_end_date = last_train_date
    print(f"Test row date: {predicted_month.date()} - Training window: {train_start_date.date()} to {train_end_date.date()} ({len(regime_train_window)} obs)")

    # Prepare training data.
    X_train = regime_train_window[FEATURES_regime].dropna()
    y_train = regime_train_window[target_regime].loc[X_train.index]

    if len(X_train) < min_data_threshold:
        print("   -> Not enough training data after dropping NAs, skipping iteration.\n")
        continue

    # Define base estimators.
    base_estimators = [
        ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
        ('svc', SVC(probability=True, random_state=42)),
        ('gnb', GaussianNB())
    ]
    final_estimator = LogisticRegression(random_state=42)
    stacking_clf = StackingClassifier(
        estimators=base_estimators,
        final_estimator=final_estimator,
        cv=5,
        n_jobs=-1
    )

    # Adjust SMOTE's k_neighbors to avoid errors with small training sets.
    smote = SMOTE(random_state=42, k_neighbors=2)

    pipeline = Pipeline([
        ('smote', smote),
        ('stacking', stacking_clf)
    ])

    try:
        pipeline.fit(X_train, y_train)
    except ValueError as e:
        print(f"   -> Error during training: {e}. Skipping iteration.\n")
        continue

    # After fitting, extract feature importances from the RandomForest base learner.
    # We assume the first estimator in the stacking classifier is the RF.
    try:
        rf_importances = pipeline.named_steps['stacking'].estimators_[0].feature_importances_
    except AttributeError:
        rf_importances = np.zeros(len(FEATURES_regime))  # fallback if not available
    feature_importances_list.append(rf_importances)

    X_test = test_row[FEATURES_regime].to_frame().T.dropna()
    if X_test.empty:
        print("   -> Test features are empty, skipping iteration.\n")
        continue

    predicted_probabilities = pipeline.predict_proba(X_test)[0]
    predicted_regime = pipeline.predict(X_test)[0]

    result = {
        'Predicted_month': predicted_month,
        'Train_Start_Date': train_start_date,
        'Train_End_Date': train_end_date,
        'Train_Count': len(X_train),
        'Predicted_Probabilities': predicted_probabilities,
        'Predicted_Regime': predicted_regime,
        'Actual_Regime': test_row[target_regime],
        'Feature_Importances': rf_importances  # store this iteration's RF importances
    }
    results_regime.append(result)
    print("   -> Regime model trained and prediction made.\n")

results_regime_df = pd.DataFrame(results_regime)
print("Headers in regime prediction results:")
print(results_regime_df.columns.tolist())

from IPython.display import display, HTML
display(HTML(results_regime_df.head(10).to_html(index=False)))

# ---------------------------
# Aggregate Feature Importances Across Rolling Windows
# ---------------------------
# If at least one iteration produced feature importances, compute their average.
if feature_importances_list:
    avg_importances = np.mean(np.vstack(feature_importances_list), axis=0)
    importance_df = pd.DataFrame({
        "Feature": FEATURES_regime,
        "Average Importance": avg_importances
    })
    # Convert to percentage format.
    importance_df["Average Importance"] = importance_df["Average Importance"].apply(lambda x: f"{x*100:.1f}%")
    print("\nAverage Feature Importances Across Rolling Windows:")
    display(HTML(importance_df.to_html(index=False)))

Test row date: 2003-02-28 - Training window: 1990-03-30 to 2003-01-31 (155 obs)
   -> Regime model trained and prediction made.

Test row date: 2003-03-31 - Training window: 1990-04-30 to 2003-02-28 (155 obs)
   -> Regime model trained and prediction made.

Test row date: 2003-04-30 - Training window: 1990-05-31 to 2003-03-31 (155 obs)
   -> Regime model trained and prediction made.

Test row date: 2003-05-30 - Training window: 1990-06-29 to 2003-04-30 (155 obs)
   -> Regime model trained and prediction made.

Test row date: 2003-06-30 - Training window: 1990-07-31 to 2003-05-30 (155 obs)
   -> Regime model trained and prediction made.

Test row date: 2003-07-31 - Training window: 1990-08-31 to 2003-06-30 (155 obs)
   -> Regime model trained and prediction made.

Test row date: 2003-08-29 - Training window: 1990-09-28 to 2003-07-31 (155 obs)
   -> Regime model trained and prediction made.

Test row date: 2003-09-30 - Training window: 1990-10-31 to 2003-08-29 (155 obs)
   -> Regime mode

Predicted_month,Train_Start_Date,Train_End_Date,Train_Count,Predicted_Probabilities,Predicted_Regime,Actual_Regime,Feature_Importances
2003-02-28,1990-03-30,2003-01-31,155,"[0.08081554248941977, 0.8803299823052678, 0.03885447520531242]",1,1,"[0.31578986815710913, 0.0973797571346361, 0.11530806902431869, 0.12349266653537389, 0.09690071933943839, 0.08436208784602314, 0.16676683196310063]"
2003-03-31,1990-04-30,2003-02-28,155,"[0.7206703436148926, 0.25555621523815975, 0.023773441146947637]",0,0,"[0.3276035931318082, 0.10045149080997344, 0.12149421241090637, 0.11896668055510493, 0.08816971802747962, 0.07969237390144142, 0.16362193116328597]"
2003-04-30,1990-05-31,2003-03-31,155,"[0.4096418116360601, 0.5564022103918064, 0.03395597797213353]",1,0,"[0.3251306949056206, 0.10707700296646365, 0.12559776751261242, 0.11213827508684224, 0.08600295125499677, 0.08199695286274226, 0.16205635541072202]"
2003-05-30,1990-06-29,2003-04-30,155,"[0.5780252072609271, 0.3974698002688898, 0.024504992470183146]",0,0,"[0.32132025105216866, 0.10858871850088163, 0.11580207116691969, 0.12222032921509757, 0.0876368376460255, 0.08037027461271869, 0.16406151780618825]"
2003-06-30,1990-07-31,2003-05-30,155,"[0.6669921661212983, 0.2761122992476658, 0.05689553463103597]",0,0,"[0.3226489202338817, 0.10917758647587023, 0.12446030614431125, 0.11169985874498174, 0.08673829175746908, 0.07978373073123082, 0.1654913059122552]"
2003-07-31,1990-08-31,2003-06-30,155,"[0.6949861416008088, 0.2737089205819804, 0.0313049378172108]",0,0,"[0.3186568783284115, 0.0993873987748043, 0.11855319648385505, 0.12544419200022563, 0.08297918132862002, 0.0861693163763092, 0.16880983670777416]"
2003-08-29,1990-09-28,2003-07-31,155,"[0.73881354924004, 0.24692333076130812, 0.014263119998651864]",0,0,"[0.3416349591978829, 0.09415299461868776, 0.10268977349788387, 0.11023945589585975, 0.07038674964939647, 0.0998983067679158, 0.18099776037237328]"
2003-09-30,1990-10-31,2003-08-29,155,"[0.8864180132007398, 0.0976447300398964, 0.015937256759363812]",0,0,"[0.3327104936365337, 0.09214983572710535, 0.10870543190099685, 0.11688156483802983, 0.07266271205137788, 0.09366510929097593, 0.18322485255498067]"
2003-10-31,1990-11-30,2003-09-30,155,"[0.9046047556002033, 0.08469453087553942, 0.010700713524257275]",0,0,"[0.32829110782046633, 0.0692804756990285, 0.132407968979002, 0.11545517941200852, 0.07408955137692574, 0.09208794178531234, 0.18838777492725667]"
2003-11-28,1990-12-31,2003-10-31,155,"[0.9114162928611883, 0.07852141629830968, 0.010062290840502088]",0,0,"[0.337562579177319, 0.07160300518670591, 0.12179487421356974, 0.1180852026435493, 0.08035083314032294, 0.08580981322761298, 0.18479369241092006]"



Average Feature Importances Across Rolling Windows:


Feature,Average Importance
VIXCLS,30.5%
CPI,7.7%
LEI,12.4%
10yr,7.7%
Financial condition,12.5%
10y-2y,9.2%
Financial Turbulance,20.1%


In [ ]:
results_regime_df

,Predicted_month,Train_Start_Date,Train_End_Date,Train_Count,Predicted_Probabilities,Predicted_Regime,Actual_Regime,Feature_Importances
0,2003-07-31,1990-03-30,2003-06-30,160,"[0.6982550877260075, 0.28511714305955416, 0.01...",0,0,"[0.3232820802561932, 0.10295713058319039, 0.11..."
1,2003-08-29,1990-04-30,2003-07-31,160,"[0.6263524232985618, 0.3614724914152827, 0.012...",0,0,"[0.3085997399062036, 0.10787074718459776, 0.12..."
2,2003-09-30,1990-05-31,2003-08-29,160,"[0.8158186011331384, 0.16931180474825797, 0.01...",0,0,"[0.3227875106314722, 0.10443014618397164, 0.12..."
3,2003-10-31,1990-06-29,2003-09-30,160,"[0.8658911675701959, 0.11411795990910316, 0.01...",0,0,"[0.32972233013854757, 0.09859367819338416, 0.1..."
4,2003-11-28,1990-07-31,2003-10-31,160,"[0.8746525664432298, 0.11115755718373854, 0.01...",0,0,"[0.3237745174038481, 0.10060721301032005, 0.12..."
...,...,...,...,...,...,...,...,...
251,2024-06-28,2011-02-28,2024-05-31,160,"[0.9192335660584467, 0.07467655893629159, 0.00...",0,0,"[0.33597366660781625, 0.06881204123961022, 0.0..."
252,2024-07-31,2011-03-31,2024-06-28,160,"[0.28470628439274, 0.7074080779059404, 0.00788...",1,0,"[0.3445938797177168, 0.07293390769952333, 0.06..."
253,2024-08-30,2011-04-29,2024-07-31,160,"[0.7458025840322633, 0.24584268934197162, 0.00...",0,0,"[0.34630820015275876, 0.07445580728047332, 0.0..."
254,2024-09-30,2011-05-31,2024-08-30,160,"[0.6359539161922997, 0.35308525153365283, 0.01...",0,0,"[0.3447188843649762, 0.07245493387811508, 0.07..."


from matplotlib import pyplot as plt
results_regime_df['Predicted_Regime'].plot(kind='hist', bins=20, title='Predicted_Regime')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
results_regime_df['Actual_Regime'].plot(kind='hist', bins=20, title='Actual_Regime')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
results_regime_df.plot(kind='scatter', x='Predicted_Regime', y='Actual_Regime', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['Predicted_month']
  ys = series['Predicted_Regime']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = results_regime_df.sort_values('Predicted_month', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Predicted_month')
_ = plt.ylabel('Predicted_Regime')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['Predicted_month']
  ys = series['Actual_Regime']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = results_regime_df.sort_values('Predicted_month', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Predicted_month')
_ = plt.ylabel('Actual_Regime')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['Predicted_month']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'Predicted_month'}, axis=1)
              .sort_values('Predicted_month', ascending=True))
  xs = counted['Predicted_month']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = results_regime_df.sort_values('Predicted_month', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Predicted_month')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['Train_Start_Date']
  ys = series['Predicted_Regime']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = results_regime_df.sort_values('Train_Start_Date', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Train_Start_Date')
_ = plt.ylabel('Predicted_Regime')

from matplotlib import pyplot as plt
results_regime_df['Predicted_Regime'].plot(kind='line', figsize=(8, 4), title='Predicted_Regime')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
results_regime_df['Actual_Regime'].plot(kind='line', figsize=(8, 4), title='Actual_Regime')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Extract true and predicted regime labels.
y_true = results_regime_df['Actual_Regime']
y_pred = results_regime_df['Predicted_Regime']

# Calculate overall accuracy.
accuracy = accuracy_score(y_true, y_pred)

# Generate a classification report.
report = classification_report(y_true, y_pred, target_names=["Regime 0", "Regime 1", "Regime 2"])

# Compute the confusion matrix.
conf_matrix = confusion_matrix(y_true, y_pred)

print("Overall Accuracy: {:.2%}".format(accuracy))
print("\nClassification Report:\n", report)
print("Confusion Matrix:")
print(conf_matrix)

Overall Accuracy: 80.86%

Classification Report:
               precision    recall  f1-score   support

    Regime 0       0.85      0.95      0.90       203
    Regime 1       0.52      0.28      0.37        46
    Regime 2       0.33      0.14      0.20         7

    accuracy                           0.81       256
   macro avg       0.57      0.46      0.49       256
weighted avg       0.77      0.81      0.78       256

Confusion Matrix:
[[193   9   1]
 [ 32  13   1]
 [  3   3   1]]
